# 03 - TFX Interactive Training Pipeline Execution

The purpose of this notebook is to interactively run the following `TFX` pipeline steps:
1. Receive hyperparameters using `hyperparam_gen` custom Python component.
2. Extract data from BigQuery using `BigQueryExampleGen` component.
3. Validate the raw data using `StatisticsGen` and `ExampleValidator` components.
4. Process the data using `Transform` component.
5. Train a custom model using `Trainer` component.
7. Evaluat and Validate the custom model using `ModelEvaluator` component.
7. Save the blessed to model registry location using `Pusher` component.
8. Upload the model to Vertex AI using `vertex_model_pusher` custom Python component

The custom components are implemented in the [tfx_pipeline/components.py](tfx_pipeline/components) module.

In [ ]:
!pip install google-cloud-aiplatform==1.3.0

## Setup

### Import libraries

In [19]:
import os
import json
import numpy as np
import tfx
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from tensorflow_transform.tf_metadata import schema_utils
import logging
!

from src.common import features
from src.model_training import data
from src.tfx_pipelines import components

logging.getLogger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')

print("Tensorflow Version:", tfx.__version__)
print("Tensorflow Version:", tf.__version__)

Tensorflow Version: 1.2.0
Tensorflow Version: 2.5.1


### Setup Google Cloud project

In [78]:
PROJECT = 'aiops-industrialization' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET = 'aiops-industrialization-bucket-ravi'  # Change to your bucket name.
SERVICE_ACCOUNT = "175728527123-compute@developer.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP projet id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn'exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
PARENT = f"projects/{PROJECT}/locations/{REGION}"
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)
print("Vertex API Parent URI:", PARENT)

Project ID: aiops-industrialization
Region: us-central1
Bucket name: aiops-industrialization-bucket-ravi
Service Account: 175728527123-compute@developer.gserviceaccount.com
Vertex API Parent URI: projects/aiops-industrialization/locations/us-central1


### Set configurations

In [80]:
VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'

WORKSPACE = f'gs://{BUCKET}/{DATASET_DISPLAY_NAME}'
RAW_SCHEMA_DIR = 'src/raw_schema'

MLMD_SQLLITE = 'mlmd.sqllite'
ARTIFACT_STORE = os.path.join(WORKSPACE, 'tfx_artifacts')
MODEL_REGISTRY = os.path.join(WORKSPACE, 'model_registry')
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-training-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME)
print(ARTIFACT_STORE)

gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts


## Create Interactive Context

In [81]:
REMOVE_ARTIFACTS = True

if tf.io.gfile.exists(ARTIFACT_STORE) and REMOVE_ARTIFACTS:
    print("Removing previous artifacts...")
    tf.io.gfile.rmtree(ARTIFACT_STORE)
    
if tf.io.gfile.exists(MLMD_SQLLITE) and REMOVE_ARTIFACTS:
    print("Deleting previous mlmd.sqllite...")
    tf.io.gfile.rmtree(MLMD_SQLLITE)
    
print(f'Pipeline artifacts directory: {PIPELINE_ROOT}')
print(f'Local metadata SQLlit path: {MLMD_SQLLITE}')

Removing previous artifacts...
Deleting previous mlmd.sqllite...
Pipeline artifacts directory: gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline
Local metadata SQLlit path: mlmd.sqllite


In [82]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = MLMD_SQLLITE
connection_config.sqlite.connection_mode = 3 # READWRITE_OPENCREATE
mlmd_store = mlmd.metadata_store.MetadataStore(connection_config)

context = InteractiveContext(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=connection_config
)

## 1. Hyperparameter generation

In [83]:
from src.tfx_pipelines import components
hyperparams_gen = components.hyperparameters_gen(
    num_epochs=5,
    learning_rate=0.001,
    batch_size=512,
    hidden_units='64,64',
)

context.run(hyperparams_gen, enable_cache=False)

I0831 13:31:20.479867  4371 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: hyperparameters_gen
    execution_id: 1
    outputs:
        hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/hyperparameters_gen/hyperparameters/1"
        custom_properties {
          key: "name"
          value {
            string_value: "hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "hyperparameters_gen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [84]:
json.load(
    tf.io.gfile.GFile(
        os.path.join(
            hyperparams_gen.outputs['hyperparameters'].get()[0].uri, 'hyperparameters.json')
    )
)

{'num_epochs': 5,
 'batch_size': 512,
 'learning_rate': 0.001,
 'hidden_units': [64, 64]}

## 2. Data extraction

In [85]:
from src.common import datasource_utils
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen
from tfx.proto import example_gen_pb2, transform_pb2

### Extract train and eval splits

In [86]:
sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='UNASSIGNED', limit=5000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
        ]
    )
)

train_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    train_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1929: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 2
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/BigQueryExampleGen/examples/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Extract test split

In [87]:
sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='TEST', limit=1000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=1),
        ]
    )
)

test_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    test_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 3
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 3
        type_id: 16
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/BigQueryExampleGen/examples/3"
        properties {
          key: "split_names"
          value {
            string_value: "[\"test\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [88]:
print(train_example_gen.outputs['examples'].get()[0].uri)

gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/BigQueryExampleGen/examples/2


### Read sample extract tfrecords

In [89]:
#train_uri = os.path.join(train_example_gen.outputs.examples.get()[0].uri, "Split-train/*")
train_uri= os.path.join(train_example_gen.outputs['examples'].get()[0].uri,"Split-train/*")
source_raw_schema = tfdv.load_schema_text(os.path.join(RAW_SCHEMA_DIR, 'schema.pbtxt'))
raw_feature_spec = schema_utils.schema_as_feature_spec(source_raw_schema).feature_spec

def _parse_tf_example(tfrecord):
    return tf.io.parse_single_example(tfrecord, raw_feature_spec)

tfrecord_filenames = tf.data.Dataset.list_files(train_uri)
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
dataset = dataset.map(_parse_tf_example)

for raw_features in dataset.shuffle(1000).batch(3).take(1):
    for key in raw_features:
        print(f"{key}: {np.squeeze(raw_features[key], -1)}")
    print("")

dropoff_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.7 42)']
euclidean: [5750.526  3090.0115 5831.0493]
loc_cross: [b'POINT(-87.7 42)POINT(-87.7 42)' b'POINT(-87.7 42)POINT(-87.7 42)'
 b'POINT(-87.7 42)POINT(-87.7 42)']
payment_type: [b'Cash' b'Cash' b'Prcard']
pickup_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.7 42)']
tip_bin: [0 0 0]
trip_day: [ 4  4 10]
trip_day_of_week: [7 3 2]
trip_hour: [ 0 16 17]
trip_miles: [3.75 0.6  5.23]
trip_month: [1 2 2]
trip_seconds: [ 736  540 1478]



## 3. Data validation

### Import raw schema

In [90]:
schema_importer = tfx.components.common_nodes.importer_node.ImporterNode(
    source_uri=RAW_SCHEMA_DIR,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False,
    #instance_name='',
)

context.run(schema_importer)

I0831 13:32:37.578145  4371 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "src/raw_schema"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Generate statistics

In [91]:
statistics_gen = tfx.components.StatisticsGen(
    examples=train_example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 5
        type_id: 20
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/StatisticsGen/statistics/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [92]:
!rm -r {RAW_SCHEMA_DIR}/.ipynb_checkpoints/

rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory


### Validate statistics against schema

In [93]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 6
        type_id: 22
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/ExampleValidator/anomalies/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [94]:
context.show(example_validator.outputs['anomalies'])

/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## 4. Data transformation

In [95]:
_transform_module_file = 'src/preprocessing/transformations.py'

transform = tfx.components.Transform(
    examples=train_example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=_transform_module_file,
    splits_config=transform_pb2.SplitsConfig(
        analyze=['train'], transform=['train', 'eval']),
)

context.run(transform, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib
installing to /tmp/tmpzbazj7x6
running install
running install_lib
copying build/lib/transformations.py -> /tmp/tmpzbazj7x6
copying build/lib/etl.py -> /tmp/tmpzbazj7x6
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpzbazj7x6/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3.7.egg-info
running inst

I0831 13:33:26.624892  4371 rdbms_metadata_access_object.cc:686] No property is defined for the Type
I0831 13:33:26.629968  4371 rdbms_metadata_access_object.cc:686] No property is defined for the Type


Processing /tmp/tmpa5rruu_u/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3-none-any.whl
Processing /tmp/tmp30dgnhkp/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3-none-any.whl
Processing /tmp/tmpn5q2aanx/tfx_user_code_Transform-0.0+de07c8431e7a29dced215501daf4f187c64541d3189d2529c8a52c51eb6c9d4d-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 7
        type_id: 24
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Transform/transform_graph/7"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Transform/transformed_examples/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 9
        type_id: 25
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Transform/updated_analyzer_cache/7"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 10
        type_id: 18
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Transform/pre_transform_schema/7"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
  

### Read sample transformed tfrecords

In [96]:
transformed_train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, "Split-train/*")
#transformed_train_uri = 'gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Transform/transformed_examples/7/train/transformed_examples-00000-of-00001.gz'
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

tft_output = tft.TFTransformOutput(transform_graph_uri)
transform_feature_spec = tft_output.transformed_feature_spec()

for input_features, target in data.get_dataset(
    transformed_train_uri, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} ({input_features[key].dtype}): {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

dropoff_grid_xf (<dtype: 'int64'>): [0, 0, 0]
euclidean_xf (<dtype: 'float32'>): [0.332584410905838, -0.8396559357643127, 0.7885392904281616]
loc_cross_xf (<dtype: 'int64'>): [0, 0, 0]
payment_type_xf (<dtype: 'int64'>): [0, 1, 0]
pickup_grid_xf (<dtype: 'int64'>): [0, 0, 0]
trip_day_of_week_xf (<dtype: 'int64'>): [5, 2, 4]
trip_day_xf (<dtype: 'int64'>): [4, 6, 26]
trip_hour_xf (<dtype: 'int64'>): [20, 2, 1]
trip_miles_xf (<dtype: 'float32'>): [0.4182177186012268, -0.3343789875507355, 0.25097396969795227]
trip_month_xf (<dtype: 'int64'>): [0, 0, 0]
trip_seconds_xf (<dtype: 'float32'>): [-0.09655807912349701, 0.013364498503506184, -0.06907743215560913]
target: [0, 1, 0]


In [ ]:
#!pip install tfx
#import tfx
#help(tfx.version)
#!pip install -i https://pypi-nightly.tensorflow.org/simple --pre tfx
#!pip install -U tfx
from tfx import v1 as tfx
tfx.__version__

## 5. Model training

In [97]:
import tfx
from tfx.components.base import executor_spec
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.experimental import latest_artifacts_resolver
from tfx.dsl.experimental import latest_blessed_model_resolver

### Get the latest model to warm start

In [98]:
latest_model_resolver = Resolver(
    strategy_class=latest_artifacts_resolver.LatestArtifactsResolver,
    latest_model=tfx.types.Channel(type=tfx.types.standard_artifacts.Model)
)

context.run(latest_model_resolver, enable_cache=False)

ExecutionResult(
    component_id: Resolver
    execution_id: 8
    outputs:
        latest_model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Train the model

In [100]:
_train_module_file = 'src/model_training/runner.py'

trainer = tfx.components.Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=_train_module_file,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    base_model=latest_model_resolver.outputs['latest_model'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=tfx.proto.trainer_pb2.TrainArgs(num_steps=0),
    eval_args=tfx.proto.trainer_pb2.EvalArgs(num_steps=None),
    hyperparameters=hyperparams_gen.outputs['hyperparameters'],
)

context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying data.py -> build/lib
copying runner.py -> build/lib
copying defaults.py -> build/lib
copying task.py -> build/lib
copying trainer.py -> build/lib
copying model.py -> build/lib
copying exporter.py -> build/lib
installing to /tmp/tmp3xrdgvbj
running install
running install_lib
copying build/lib/task.py -> /tmp/tmp3xrdgvbj
copying build/lib/exporter.py -> /tmp/tmp3xrdgvbj
copying build/lib/defaults.py -> /tmp/tmp3xrdgvbj
copying build/lib/data.py -> /tmp/tmp3xrdgvbj
copying build/lib/trainer.py -> /tmp/tmp3xrdgvbj
copying build/lib/runner.py -> /tmp/tmp3xrdgvbj
copying build/lib/model.py -> /tmp/tmp3xrdgvbj
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manif

Processing /tmp/tmp3id3rv9d/tfx_user_code_Trainer-0.0+ad94d6efc657a1377b407734494c932dc5805d4dc172d43ba973e869086ae261-py3-none-any.whl


2021-08-31 13:40:27.412336: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-31 13:40:27.412383: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-31 13:40:27.412443: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/5
      2/Unknown - 1s 1s/step - loss: 0.6731 - accuracy: 0.88 - 1s 257ms/step - loss: 0.6618 - accuracy: 0.8945

2021-08-31 13:40:32.396655: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-31 13:40:32.396707: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-31 13:40:32.736473: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-31 13:40:32.740155: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-31 13:40:34.401020: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Trainer/model_run/10/train/plugins/profile/2021_08_31_13_40_33
2021-08-31 13:40:34.645762: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipe

      3/Unknown - 5s 2s/step - loss: 0.6497 - accuracy: 0.9004   

2021-08-31 13:40:36.204296: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Trainer/model_run/10/train/plugins/profile/2021_08_31_13_40_33Dumped tool data for xplane.pb to gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Trainer/model_run/10/train/plugins/profile/2021_08_31_13_40_33/ravi-tensorflow-2-3-20210824-140339.xplane.pb
Dumped tool data for overview_page.pb to gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Trainer/model_run/10/train/plugins/profile/2021_08_31_13_40_33/ravi-tensorflow-2-3-20210824-140339.overview_page.pb
Dumped tool data for input_pipeline.pb to gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training

7/7 [==============================] - 7s 944ms/step - loss: 0.6067 - accuracy: 0.8917 - val_loss: 0.5374 - val_accuracy: 0.8818
Epoch 2/5
7/7 [==============================] - ETA: 1s - loss: 0.5276 - accuracy: 0.88 - 1s 89ms/step - loss: 0.4789 - accuracy: 0.8898 - val_loss: 0.4356 - val_accuracy: 0.8809
Epoch 3/5
7/7 [==============================] - ETA: 0s - loss: 0.4192 - accuracy: 0.88 - 1s 74ms/step - loss: 0.3809 - accuracy: 0.8923 - val_loss: 0.3603 - val_accuracy: 0.8809
Epoch 4/5
7/7 [==============================] - ETA: 0s - loss: 0.3293 - accuracy: 0.89 - 1s 75ms/step - loss: 0.3087 - accuracy: 0.8945 - val_loss: 0.3111 - val_accuracy: 0.8799
Epoch 5/5
7/7 [==============================] - ETA: 1s - loss: 0.2920 - accuracy: 0.88 - 1s 99ms/step - loss: 0.2642 - accuracy: 0.8940 - val_loss: 0.2797 - val_accuracy: 0.8799


ExecutionResult(
    component_id: Trainer
    execution_id: 10
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 17
        type_id: 28
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Trainer/model/10"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 18
        type_id: 29
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Trainer/model_run/10"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## 6. Model evaluation

### Get the latest blessed model for model validation.

In [101]:
blessed_model_resolver = Resolver(
    strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=tfx.types.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=tfx.types.Channel(type=tfx.types.standard_artifacts.ModelBlessing)
)

context.run(blessed_model_resolver, enable_cache=False)

ExecutionResult(
    component_id: Resolver
    execution_id: 11
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Evaluate and validate the model against the baseline model.

In [102]:
from tfx.components import Evaluator

In [103]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_tf_example',
            label_key=features.TARGET_FEATURE_NAME,
            prediction_key='probabilities')
    ],
    slicing_specs=[
        tfma.SlicingSpec(),
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[   
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.7}),
                        # Change threshold will be ignored if there is no
                        # baseline model resolved from MLMD (first run).
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10}))),
        ])
    ])


evaluator = Evaluator(
    examples=test_example_gen.outputs['examples'],
    example_splits=['test'],
    model=trainer.outputs['model'],
    baseline_model=blessed_model_resolver.outputs['model'],
    eval_config=eval_config,
    schema=schema_importer.outputs['result']
)

context.run(evaluator, enable_cache=False)

I0831 13:42:07.718508  4371 rdbms_metadata_access_object.cc:686] No property is defined for the Type
I0831 13:42:07.723235  4371 rdbms_metadata_access_object.cc:686] No property is defined for the Type
ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).
Exception ignored in: <function CapturableResource.__del__ at 0x7f23739cd050>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/training/tracking/tracking.py", line 277, in __del__
    self._destroy_resource()
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
 

ExecutionResult(
    component_id: Evaluator
    execution_id: 12
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 19
        type_id: 31
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Evaluator/evaluation/12"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 20
        type_id: 32
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Evaluator/blessing/12"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Trainer/model/10"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 17
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [104]:
evaluation_results = evaluator.outputs['evaluation'].get()[0].uri
print("validation_ok:", tfma.load_validation_result(evaluation_results).validation_ok, '\n')

for entry in list(tfma.load_metrics(evaluation_results))[0].metric_keys_and_values:
    value = entry.value.double_value.value
    if value:
        print(entry.key.name, ":", round(entry.value.double_value.value, 3))

validation_ok: True 

accuracy : 0.894
loss : 0.723
binary_accuracy : 0.894
example_count : 1000.0


## 7. Model pushing

In [105]:
exported_model_location = os.path.join(MODEL_REGISTRY, MODEL_DISPLAY_NAME)

push_destination=tfx.proto.pusher_pb2.PushDestination(
    filesystem=tfx.proto.pusher_pb2.PushDestination.Filesystem(
        base_directory=exported_model_location,
    )
)

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=push_destination
)

context.run(pusher, enable_cache=False)

I0831 13:45:07.698115  4371 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: Pusher
    execution_id: 13
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 21
        type_id: 34
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/Pusher/pushed_model/13"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/model_registry/chicago-taxi-tips-classifier-v01/1630417507"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1630417507"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 34
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## 8. Model Upload to AI Platform

In [110]:
serving_runtime = 'tf2-cpu.2-4'
serving_image_uri = f"gcr.io/cloud-aiplatform/prediction/{serving_runtime}:latest"

vertex_model_uploader = components.vertex_model_uploader(
    #project=PROJECT,
    project='aiops-industrialization',
    region=REGION,
    model_display_name=MODEL_DISPLAY_NAME,
    pushed_model_location=exported_model_location,
    serving_image_uri=serving_image_uri,
    explanation_config=''
)

context.run(vertex_model_uploader, enable_cache=False)

ExecutionResult(
    component_id: vertex_model_uploader
    execution_id: 15
    outputs:
        uploaded_model: Channel(
            type_name: File
            artifacts: [Artifact(artifact: id: 23
        type_id: 36
        uri: "gs://aiops-industrialization-bucket-ravi/chicago-taxi-tips/tfx_artifacts/chicago-taxi-tips-classifier-v01-training-pipeline/vertex_model_uploader/uploaded_model/15"
        custom_properties {
          key: "model_uri"
          value {
            string_value: "projects/175728527123/locations/us-central1/models/8287529311943000064"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "uploaded_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "vertex_model_uploader"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 36
        name: "File"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [111]:
vertex_model_uploader.outputs['uploaded_model'].get()[0].get_string_custom_property('model_uri')

'projects/175728527123/locations/us-central1/models/8287529311943000064'

In [113]:
statistics_gen = tfx.components.StatisticsGen(examples=train_example_gen.outputs['examples']).with_id("StatisticsGen")

In [114]:
statistics=statistics_gen.outputs['statistics'],

In [115]:
print(statistics)

(Channel(
    type_name: ExampleStatistics
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
),)


In [116]:
schema=schema_importer.outputs['result']

In [117]:
print(schema)

Channel(
    type_name: Schema
    artifacts: [Artifact(artifact: id: 4
type_id: 18
uri: "src/raw_schema"
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.2.0"
  }
}
, artifact_type: id: 18
name: "Schema"
)]
    additional_properties: {}
    additional_custom_properties: {}
)
